1.仿照课件示例的GAN生成网络代码，实现Fashion_mnist数据集的条件生成对抗网络CGAN。<BR>
    （对于感觉困难同学，可以降级选择完成GAN练习，计80%）

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import PIL
from keras import Sequential,layers
from keras.layers import Dense,LeakyReLU,Conv2DTranspose,BatchNormalization,Reshape
from keras.layers import Conv2D,Dropout,Flatten
from keras.optimizers import Adam
import time
import glob
from IPython import display

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

train_images = train_images.reshape(train_images.shape[0],28,28,1).astype('float32')
train_images  = (train_images-127.5) / 127.5

train_labels = tf.one_hot(train_labels, depth=10)
train_labels = tf.cast(train_labels, tf.float32)

In [3]:
buffer_size = 60000
batch_size = 256
train_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_labels)).shuffle(buffer_size).batch(batch_size)

In [4]:
# generator
def make_generator_model():
    model = Sequential()
    model.add(Dense(7*7*256,use_bias=False,input_shape=(110,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Reshape((7,7,256)))
    assert model.output_shape == (None,7,7,256)

    model.add(Conv2DTranspose(128,(5,5),strides=(1,1),padding='same',use_bias=False))
    assert model.output_shape == (None,7,7,128)
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(64,(5,5),strides=(2,2),padding='same',use_bias=False))
    assert model.output_shape == (None,14,14,64)
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(1,(5,5),strides=(2,2),padding='same',use_bias=False,activation='tanh'))
    assert model.output_shape == (None,28,28,1)
    
    return model

In [5]:
generator = make_generator_model()
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12544)             1379840   
                                                                 
 batch_normalization (BatchN  (None, 12544)            50176     
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 12544)             0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 7, 7, 128)        819200    
 nspose)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 128)        5

In [6]:
# discriminator
def make_discriminator_model():
    model = Sequential()
    model.add(Conv2D(64,(5,5),strides=(2,2),padding='same',input_shape=[28,28,2]))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Conv2D(128,(5,5),strides=(2,2),padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(1))

    return model

In [7]:
discriminator = make_discriminator_model()
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        3264      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)             

In [8]:
noise = tf.random.normal([1,110])
generated_image = generator(noise,training=False)
plt.imshow(generated_image[0,:,:,0],cmap='gray')

: 

: 

In [ ]:
noise_label = tf.random.normal([1,10])
noise_label = tf.concat([noise_label,noise_label,tf.zeros([1,8],dtype='float32')],1)
noise_label = tf.reshape(noise_label,[1,28,1])
noise_label = noise_label * tf.ones([1,28,28],dtype='float32')
noise_label = tf.reshape(noise_label,[1,28,28,1])
rand_input = tf.concat([generated_image,noise_label],3)

decision = discriminator(rand_input)
decision

In [ ]:
# loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output),fake_output)

def discriminator_loss(real_output,fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
# optimizer
generator_optimizer = Adam(1e-4)
discriminator_optimizer = Adam(1e-4)

In [ ]:
epochs = 100
noise_dim = 100
num_examples_to_generate = 100

seed = tf.random.normal([num_examples_to_generate,noise_dim])

labels = [i%10 for i in range(num_examples_to_generate)]
labels = tf.one_hot(labels,depth=10)
labels = tf.cast(labels,tf.float32)

seed = tf.concat([seed,labels],1)

print(seed.shape)

In [ ]:
def generate_images(model,test_input):
    predictions = model(test_input,training=False)

    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow(predictions[i,:,:,0]*127.5+127.5, cmap='gray')
        plt.axis('off')

    plt.show()

In [ ]:
@tf.function
def train_step(data_batch):
    images = data_batch[0]
    labels = data_batch[1]
    batch_size = images.shape[0]

    labels_input = tf.concat([labels,labels,tf.zeros([batch_size,8],dtype='float32')],1)
    labels_input = tf.reshape(labels_input,[batch_size,28,1])
    labels_input = labels_input * tf.ones([batch_size,28,28],dtype='float32')
    labels_input = tf.reshape(labels_input,[batch_size,28,28,1])

    noise = tf.random.normal([batch_size,noise_dim])
    noise_input = tf.concat([noise,labels],1)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise_input,training=True)
        image_input = tf.concat([images,labels_input],3)
        gen_input = tf.concat([generated_images,labels_input],3)

        real_output = discriminator(image_input,training=True)
        fake_output = discriminator(gen_input,training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output,fake_output)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss,disc_loss

In [ ]:
def train(dataset,epochs):
    for epoch in range(epochs):
        start = time.time()
        for i,image_batch in enumerate(dataset):
            g,d = train_step(image_batch)
#             print('batch %d, gen_loss %f, disc_loss %f' % (i,g.numpy(),d.numpy()))

        display.clear_output(wait=True)
#         generate_images(generator,seed)

        # if (epoch+1)%5==0:
        #     checkpoint.save(file_prefix=checkpoint_prefix)

        print('time for eopch{} is {}s'.format(epoch+1,time.time()-start))

In [ ]:
%%time
train(train_dataset,epochs)

In [ ]:
test_input = tf.random.normal([16,110])
generate_images(generator,test_input)